# importar as bibliotecas

In [ ]:
import spacy
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# executar as funções

In [ ]:
def searchSimilar(phrase, Comments) -> list:
	filterdData = []
	
	nlp = spacy.load("pt_core_news_sm")
	corpus_old = (list(Comments['content'].values))
	corpus_old.insert(0, phrase) #concatena a phrase na base dados e coloca a phrase no inicio
    
	for i in corpus_old:
		filterdData.append(clearPhrase(i, nlp, ['PROPN', 'ADJ', 'NOUN', 'VERB'])) #eu faço uma limpeza nos comentários, deixando apenas Pronomes pessoais, adjetivos, substantivos e verbos 
	#filterdData = corpus_old	
	processedData = TfidfVectorizer().fit_transform(filterdData) #transformo a base de dados em vetores de números TF-IDF
	print(corpus_old[0])
	[lenght, M] = processedData.shape #pego o tamanho da base de dados    
	similaritiesList = []    
	for i in range(1, lenght):
		s = cosine_similarity(processedData[0],processedData[i]) #o comentário processedData[0] é a frase digitada pelo usuário, daí preciso computar a distancia dessa frase com todos os comentários da base de dados
		if(s != 0): #se a similaridade é zero, nem adiciono o comentário na lista, pois significa que ele é 0 similar
			similaritiesList.append([s, corpus_old[i]])
		
	return similaritiesList   #retorna uma lista com o comentário e a distancia dele da frase que o usuário digitou
	
def clearPhrase(doc, nlp, pos_tag):
	if(type(doc) != spacy.tokens.doc.Doc):
		doc = nlp(doc)
	return ' '.join([str(t.lemma_) for t in doc if t.pos_ in pos_tag]) 

def addHours(minDate:str,maxDate:str)-> (str,str):
	minDate = minDate + "00:00:00"
	maxDate = maxDate + "23:59:59"

	return minDate,maxDate
def getComments(originPath:str,minDate:str = None,maxDate:str = None,versions = None) -> pandas.DataFrame:
	
	commentsFile = pandas.read_csv(originPath)

	if minDate != None and maxDate != None:
		minDate,maxDate = addHours(minDate,maxDate)

	for i in range(0, len(commentsFile)):
		if commentsFile['date'][i] < minDate or commentsFile['date'][i] > maxDate or commentsFile['version'][i] not in versions:
			#print("droped",commentsFile['version'][i])
			commentsFile = commentsFile.drop([i])
	return commentsFile

def sortbyThumbs(file:pandas.DataFrame) -> pandas.DataFrame:
	file = file.sort_values(by=['thumbs'])
	return file

def limparComentarios(df):
    nlp = spacy.load("pt_core_news_sm")
    lista = []
    for i in df['content']:
        r = clearPhrase(i, nlp, ['PROPN', 'NOUN'])
        lista.append(r)    
    return lista

def extrairNgrams(corpus, AA, BB):
    c_vec = CountVectorizer(ngram_range=(AA,BB))
    ngrams = c_vec.fit_transform(corpus)
    count_values = ngrams.toarray().sum(axis=0)
    vocab = c_vec.vocabulary_
    df_ngram = pandas.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})
    
    return df_ngram


# coletar os comentários

In [ ]:
database = getComments("./General_Data.csv","2021-09-01","2021-10-01","3.0.0")

# buscar por comentários similares

In [ ]:
similarities_list = searchSimilar('nao consigo recuperar minha senha', database)

In [ ]:
print(sorted(similarities_list))

In [ ]:
d = limparComentarios(database)

In [ ]:
n = extrairNgrams(d, 1, 1)

In [ ]:
print(n)